**<font size=5 color=black>GBDT</font>**

**1** 介绍GBDT(Gradient Boosting Decision Tree)的基本原理

**2** 手动实现GBDT回归算法的基本框架

**3** 分别以一元和多元回归任务为例，对比本文算法与sklearn的结果

In [1]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import load_boston
import numpy as np

**<font size=4 color=black>1 GBDT算法</font>**

**<font size=3.5 color=black>1.1 思想</font>**

在每次迭代过程中，通过**拟合负梯度**来学习一个弱学习器。

一个简单例子：<font size=3.5 color=blue>已知小明的年龄是30岁，但GBDT模型不知道小明的年龄，那么它如何去预测小明的年龄？</font>

1）首先，GBDT用第一个学习器来拟合，预测结果为20岁，发现残差为10岁；

2）接着，用第二个学习器来拟合第一个学习器的残差，预测结果为6岁，发现残差为4岁；

3）然后，用第三个学习器来拟合第二个学习器的残差，预测结果为3岁，发现残差为1岁。

假设这时GBDT没有其他学习器了，或者误差大小已满足我们的要求，那么GBDT最后输出的预测结果为20+6+3=29岁。

**<font size=3.5 color=red>说明：</font>**

上述例子仅用于粗略理解GBDT的学习过程，实际上只有GBDT模型的损失函数为平方损失时，才能按上述方法进行学习。

**<font size=3.5 color=black>1.2 流程</font>**

GBDT可解释为以**CART回归树**为基学习器的加法模型，损失函数通常为平方损失、绝对损失等，学习算法为前向分步

算法的学习方法。可用于回归和分类问题，本文仅以回归问题为例，给出GBDT算法的基本流程。
$\rule[1pt]{23cm}{0.1em}$
**输入**：训练数据集$D={(x_1, y_1),(x_2, y_2),...,(x_N, y_N)},x_i \in R^n,y_i \in R$；损失函数为$L(y, f(x))$；<br>
**输出**：回归树$\hat{f}(x)$

**（1）**初始化
$$f_0(x) = \mathop{argmin} \limits_c \sum_{i=1}^{N}L(y_i, c)\tag{1}$$

**（2）**假设进行$m$轮训练，$m=1,2,...,M$

**（2a）**对$i=1,2,...,N$，计算负梯度
$$r_{mi}=-\big[\frac{\partial L(y_i,f(x_i))}{\partial f(x_i)}\big]_{f(x)=f_{m-1}(x)}\tag{2}$$

**（2b）**对${(x_1, r_{m1}),(x_2, r_{m2}),...,(x_N, r_{mN})}$拟合一个回归树，得到叶节点区域$R_{mj}$，$j=1,2,...,J$

**（2c）**对$j=1,2,...,J$，计算每个叶节点的预测值，即
$$c_{mj}=\mathop{argmin} \limits_{c} \sum_{x_i \in R_{mj}} L(y_i,f_{m-1}(x_i)+c)\tag{3}$$

**（2d）**更新
$$f_m(x)=f_{m-1}(x)+\sum_{j=1}^{J}c_{mj}I(x \in R_{mj})\tag{4}$$

**（3）**得到回归树
$$\hat{f}(x)=f_M(x)=f_0(x)+\sum_{m=1}^{M}\sum_{j=1}^{J}c_{mj}I(x \in R_{mj})\tag{5}$$
$\rule[1pt]{23cm}{0.1em}$

**<font size=3.5 color=red>几点说明：</font>**

**（1）GBDT的本质在于将划分（2b）和取值（2c）分为两步进行**，即先利用回归树拟合负梯度得到叶节点区域，然后再

根据式(3)计算每个叶节点的输出值。那为什么要这么做？

在参数空间，对于损失函数$L(y,\theta)$我们采用梯度下降方法按如下公式不断更新参数$\theta$的值，来逐渐逼近$min(L(y,\theta))$
$$\theta_{k+1}=\theta_k-\eta \frac{\partial L(y,\theta_k)}{\partial \theta_k}\tag{6}$$
而在函数空间，对于损失函数$L(y,f_m(x))$，我们可以将$f_m(x)$看作一个参数，类比可以得到下面的更新公式：
$$f_m(x)=f_{m-1}(x)-\eta \frac{\partial L(y,f_{m-1}(x))}{\partial f_{m-1}(x)}\tag{7}$$
令$T_m(x)=-\eta \frac{\partial L(y,f_{m-1}(x))}{\partial f_{m-1}(x)}$，则式(7)变为
$$f_m(x)=f_{m-1}(x)+T_m(x)\tag{8}$$

这就是我们的提升树模型。

在进行第m轮的训练时，到(2b)这一步根据式(2)用回归树对负梯度进行拟合，可以看到式(2)和式(8)的$T_m(x)$有一个$\eta$的差别，只有当$\eta$为固定值1时，二者等价。但在使用梯度下降法的时候，我们清楚对于固定步长的$\eta$，优化效果可能不佳，因此(2c)这一步，可以理解为起到了对每个叶节点的输出值自适应步长$\eta$的作用，使得当前模型$f_m(x)$对应的损失$L(y,f_m(x))$最小。

**（2）平方损失函数**$L(y,f(x))=\frac{1}{2}(y-f(x))^2$

初始化：$f_0(x)=\frac{1}{N}\sum_{i=1}^{N}y_i$，即$y$的均值

负梯度：$r_{mi}=y_i-f_{m-1}(x_i)$，即模型$f_{m-1}$拟合数据的残差

叶节点的输出值：$c_{mj}=\frac{1}{N}\sum_{i=1}^{N}(y_i-f_{m-1}(x_i))$，即模型$f_{m-1}$拟合数据的残差的均值

**（3）绝对损失函数**$L(y,f(x))=|y-f(x)|$

初始化：$f_0(x)=median(y_i)$，即$y$的中位数

负梯度：$r_{mi}=sign(y_i-f_{m-1}(x_i))$，只有1和-1两种取值

叶节点的输出值：$c_{mj}=median(y_i-f_{m-1}(x_i))$，即模型$f_{m-1}$拟合数据的残差的中位数

<font color=red>*注：本文主要阐述了GBDT的一些基础理论，更多关于GBDT的知识见参考。*</font>

**<font size=4 color=black>2 手动实现GBDT回归算法</font>**

（1）本章重点在于理解GBDT算法的学习流程，对于基回归器的实现借助于[sklearn.ensemble.DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

（2）为方便后续案例测试结果的对比，实现过程的一些细节适当参考了sklearn中的实现源码

（3）为简化代码，本章仅对一些必要参数进行设置，仅通过具体的实现过程帮助理解算法，并不追求最终的预测精度

In [2]:
class MyGradientBoostingRegressor(object):
    """GBDT回归器   
    Parameters
    ----------
    loss: str 
        损失函数 本文算法支持{'ls', 'lad'}，默认为'ls'
    learning_rate: float
        学习率 防止过拟合，默认为1.0 可以参考sklearn文档和源码了解该参数的使用
    n_estimators: int
        基回归器的个数
    criterion: str
        决策树的分裂标准 本文算法支持{'mse','friedman_mse'}，默认为'mse'
    max_depth: int
        决策树的最大深度

    Attributes
    ----------
    init_raw_predictions: ndarray of shape (n_samples,)
        初始预测值
    estimators_: list of DecisionTreeRegressor of shape (n_estimators,)
        每一轮训练的基回归器
    """

    def __init__(self, loss='ls', learning_rate=1.0, n_estimators=3,
                 criterion='mse', max_depth=1):
        self.loss = loss
        self.learning_rate = learning_rate
        self.n_estimators = n_estimators
        self.criterion = criterion
        self.max_depth = max_depth
        self.init_raw_predictions = None
        self.estimators_ = []

    def fit(self, X, y):
        """训练回归器
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features) 样本向量
        y: ndarray of shape (n_samples,) 目标值
        """
        # 计算初始预测值
        raw_predictions = self.get_init_raw_predictions(y)
        self.init_raw_predictions = raw_predictions

        for i in range(self.n_estimators):
            # 构建决策树回归器
            estimator = DecisionTreeRegressor(criterion=self.criterion,
                                              max_depth=self.max_depth,
                                              random_state=42)
            # 计算负梯度
            neg_grad = self.negative_gradient(y, raw_predictions)
            # 拟合负梯度
            estimator.fit(X, neg_grad)
            # 更新叶节点预测值
            if self.loss == 'ls':
                y = y - raw_predictions
                raw_predictions = self.learning_rate * estimator.predict(X)
            elif self.loss == 'lad':
                y = y - raw_predictions
                diff = y
                raw_predictions = self.update_teminal_predictions(
                    estimator, X, diff)
            # 存储学习完成的回归器
            self.estimators_.append(estimator)

    def predict(self, X):
        """预测回归值
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features) 样本向量

        Returns
        -------
        out: ndarray of shape (n_samples,) 回归值
        """
        pred = self.init_raw_predictions[:X.shape[0]]
        for estimator in self.estimators_:
            pred += self.learning_rate * estimator.predict(X)
        return pred

    def negative_gradient(self, y, raw_predictions):
        """计算负梯度
        Parameters
        ----------
        y: ndarray of shape (n_samples,) 目标值
        raw_predictions: ndarray of shape (n_samples,) 目标值的预测值

        Returns
        -------
        out: ndarray of shape (n_samples,) 负梯度
        """
        if self.loss == 'ls':
            return y - raw_predictions
        elif self.loss == 'lad':
            return 2 * (y - raw_predictions > 0) - 1

    def get_init_raw_predictions(self, y):
        """计算初始预测值
        Parameters
        ----------
        y: ndarray of shape (n_samples,) 目标值

        Returns
        -------
        out: ndarray of shape (n_samples,) 目标值的初始预测值
        """
        if self.loss == 'ls':
            return np.repeat(np.mean(y), y.shape[0])
        elif self.loss == 'lad':
            return np.repeat(np.median(y), y.shape[0])

    def update_teminal_predictions(self, estimator, X, diff):
        """更新决策树叶节点的输出值。对于本文算法而言，当loss='ls'时，
        不需要更新叶节点的输出值，仅loss='lad'时需要更新叶节点的输出值。
        Parameters
        ----------
        estimator: class 决策树回归器
        X: ndarray of shape (n_samples, n_features) 样本向量
        diff: ndarray of shape (n_samples,) diff = y - raw_predictions

        Returns
        -------
        out: ndarray of shape (n_samples,) 预测值
        """
        leaves_index = estimator.apply(X)  # 获取每个样本落入的叶节点id
        for i in np.unique(leaves_index):
            ind = np.where(leaves_index == i)[0]
            # 计算中位数 参考sklearn中个数为偶数时中位数取较小的那个
            sort_diff = np.sort(diff[ind])
            med_ind = (sort_diff.shape[0] - 1) // 2 
            # 更新叶节点的输出值
            estimator.tree_.value[i] = sort_diff[med_ind]
        return self.learning_rate * estimator.predict(X)

**<font size=4 colot=black>3 案例</font>**

**<font size=3.5 colot=black>3.1 一元回归</font>**

In [3]:
# 数据准备
X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]).reshape(-1, 1)
y = np.array([5.56, 5.70, 5.91, 6.40, 6.80, 7.05, 8.90, 8.70, 9.00, 9.05])

In [4]:
# 本文算法
myreg = MyGradientBoostingRegressor(loss='lad', max_depth=3)
myreg.fit(X, y)

print('本文算法')
print('-------')
print('集成回归器预测结果：\n', myreg.predict(X))
print('基回归器1预测结果：\n', myreg.estimators_[0].predict(X))
print('基回归器2预测结果：\n', myreg.estimators_[1].predict(X))
print('基回归器3预测结果：\n', myreg.estimators_[2].predict(X))

本文算法
-------
集成回归器预测结果：
 [5.56 5.56 5.91 6.4  6.4  8.7  8.7  8.7  9.   9.05]
基回归器1预测结果：
 [-1.015 -1.015 -1.015 -1.015 -1.015  1.975  1.975  1.975  1.975  1.975]
基回归器2预测结果：
 [-0.21 -0.21 -0.21  0.49  0.49 -0.2  -0.2  -0.2   0.1   0.1 ]
基回归器3预测结果：
 [-0.14 -0.14  0.21  0.    0.    0.    0.    0.    0.    0.05]


In [5]:
# sklearn
reg = GradientBoostingRegressor(loss='lad', random_state=42, n_estimators=3,
                                learning_rate=1.0, max_depth=3, criterion='mse')
reg.fit(X, y)

print('sklearn')
print('-------')
print('集成回归器预测结果：\n', reg.predict(X))
print('基回归器1预测结果：\n', reg.estimators_[0][0].predict(X))
print('基回归器2预测结果：\n', reg.estimators_[1][0].predict(X))
print('基回归器3预测结果：\n', reg.estimators_[2][0].predict(X))

sklearn
-------
集成回归器预测结果：
 [5.56 5.56 5.91 6.4  6.4  8.7  8.7  8.7  9.   9.05]
基回归器1预测结果：
 [-1.015 -1.015 -1.015 -1.015 -1.015  1.975  1.975  1.975  1.975  1.975]
基回归器2预测结果：
 [-0.21 -0.21 -0.21  0.49  0.49 -0.2  -0.2  -0.2   0.1   0.1 ]
基回归器3预测结果：
 [-0.14 -0.14  0.21  0.    0.    0.    0.    0.    0.    0.05]


可以看出，对于一元回归问题，本文算法和sklearn的结果一致。

**<font size=3.5 colot=black>3.2 多元回归</font>**

In [6]:
# 波士顿房价数据集
boston = load_boston()
X2 = boston.data
y2 = boston.target
X2_test = X2[[0, 100, 200], :]  # 为方便显示，选取三个样本用于预测

In [7]:
# 本文算法
myreg2 = MyGradientBoostingRegressor(loss='ls', max_depth=3)
myreg2.fit(X2, y2)

print('本文算法')
print('-------')
print('集成回归器预测结果：\n', myreg2.predict(X2_test))
print('基回归器1预测结果：\n', myreg2.estimators_[0].predict(X2_test))
print('基回归器2预测结果：\n', myreg2.estimators_[1].predict(X2_test))
print('基回归器3预测结果：\n', myreg2.estimators_[2].predict(X2_test))

本文算法
-------
集成回归器预测结果：
 [28.65131748 20.89789538 33.39272232]
基回归器1预测结果：
 [ 0.37239368  0.37239368 10.81603089]
基回归器2预测结果：
 [ 5.45614711 -0.36700372 -0.24608527]
基回归器3预测结果：
 [ 0.28997038 -1.6403009   0.28997038]


In [8]:
# sklearn
reg2 = GradientBoostingRegressor(loss='ls', random_state=42, n_estimators=3,
                                 learning_rate=1.0, max_depth=3, criterion='mse')
reg2.fit(X2, y2)

print('sklearn')
print('-------')
print('集成回归器预测结果：\n', reg2.predict(X2_test))
print('基回归器1预测结果：\n', reg2.estimators_[0][0].predict(X2_test))
print('基回归器2预测结果：\n', reg2.estimators_[1][0].predict(X2_test))
print('基回归器3预测结果：\n', reg2.estimators_[2][0].predict(X2_test))

sklearn
-------
集成回归器预测结果：
 [28.65131748 20.89789538 33.39272232]
基回归器1预测结果：
 [ 0.37239368  0.37239368 10.81603089]
基回归器2预测结果：
 [ 5.45614711 -0.36700372 -0.24608527]
基回归器3预测结果：
 [ 0.28997038 -1.6403009   0.28997038]


可以看出，对于多元回归问题，本文算法和sklearn的结果一致。

**<font color=black size=4>参考</font>**

1 李航. (2012) 统计学习方法. 清华大学出版社, 北京.

2 [Friedman J H . Greedy Function Approximation: A Gradient Boosting Machine[J]. Annals of Statistics, 2001, 29(5):1189-1232.](https://statweb.stanford.edu/~jhf/ftp/trebst.pdf)

3 [sklearn.ensemble.GradientBoostingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html)

4 [sklearn.ensemble.DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

5 [sklearn-criterion](https://github.com/scikit-learn/scikit-learn/blob/main/sklearn/tree/_criterion.pyx)